In [1]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
div.container{width:80% !important;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
span.toc-item-num{display:none;}
div.CodeMirror {font-family:Consolas}
div.input {font-family:Consolas}
</style>
"""))

# 인코더 LSTM과 디코더 LSTM으로 스마트번역기 만들기

In [2]:
# 1. 패키지 수입
import numpy as np
import pandas as pd
from time import time

from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# 하이퍼 파라미터 (정확도, 속도에 영향을 미치는 값)
MY_HIDDEN = 128
MY_EPOCH  = 500

In [3]:
# 2. 번역 데이터 불러오기
raw = pd.read_csv('trans_dataset/translate.csv', header=None)
eng_kor = raw.values.tolist()  # 데이터 프레임 -> list 변환
print(len(eng_kor))
print(eng_kor[:3])

110
[['cold', '감기'], ['come', '오다'], ['cook', '요리']]


In [4]:
# 3. 알파벳과 한글 리스트 만들기
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']  # 리스트
# {c:i for i, c in enumerate(e_alpha)}  # 딕셔너리
# for idx, c in enumerate(e_alpha):
#     print(c, idx)
k_alpha = pd.read_csv('trans_dataset/korean.csv', header=None)[0].tolist()  # 1개 열만 존재하므로 [0]만 추출하면 시리즈로 받아옴
alpha = e_alpha + k_alpha  # 리스트 + 리스트
print('영어와 한글 알파벳 : ', alpha)
alpha_total_size = len(alpha)
print('전체 알파벳 갯수(원핫인코딩할 size) : ', alpha_total_size)
print('영어 알파벳 갯수 : ', len(e_alpha))
print('한글 글자수 갯수 : ', len(k_alpha))

영어와 한글 알파벳 :  ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩할 size) :  171
영어 알파벳 갯수 :  29
한글 글자수 갯수 :  142


In [5]:
# 문자당 num을 갖는 dict 만들기
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [6]:
eng_kor[0]     # ['cold', '감기']
eng_kor[0][0]  # ['c', 'o', 'l', 'd']
print(char_to_num['c'], char_to_num['o'], char_to_num['l'], char_to_num['d'])
print([c for c in eng_kor[0][0]])
print([char_to_num[c] for c in eng_kor[0][0]])
print(char_to_num['감'], char_to_num['기'])
print([c for c in eng_kor[0][1]])
print([char_to_num[c] for c in eng_kor[0][1]])

5 17 14 6
['c', 'o', 'l', 'd']
[5, 17, 14, 6]
32 46
['감', '기']
[32, 46]


In [7]:
# 원핫인코딩 방법1 : 이 코드에서는 적용이 어려움.
# display(pd.get_dummies([5, 3, 6]))

# 원핫인코딩 방법2
to_categorical([5, 3, 6], num_classes=10)  # 0~9인덱스 원핫인코딩

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [8]:
# 원핫인코딩 방법3 : 단위행렬을 이용한 방법
np.eye(10)[[5, 3, 6]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [9]:
# 5. 인코더 입력(영어알파벳 -> 숫자로 바꾼 값을 원핫인코딩)
#    디코더 입력("S한글"을 숫자로 바꾼 값을 원핫인코딩)     
#    디코더 출력("한글E"를 숫자로 바꾼 값)
def encoding(eng_kor=eng_kor):
    enc_in = []
    dec_in = []
    dec_out = []  # [[32, 57, 1], [62, 78, 1]]
    for data in eng_kor:
        # 인코딩 입력 데이터(영어->숫자->원핫인코딩)
        eng = [char_to_num[c] for c in data[0]]  # 영어->숫자
        eng_one = np.eye(alpha_total_size)[eng]  # 숫자->원핫인코딩
        # eng_one = to_categorical(eng, num_classes=alpha_total_size)
        enc_in.append(eng_one)
        # 디코더 입력 데이터('S한글'->숫자->원핫인코딩)
        kor = [char_to_num[c] for c in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        # 디코더 출력 데이터('한글E'->숫자)
        kor = [char_to_num[c] for c in data[1]+'E']
        dec_out.append(kor)
    return enc_in, dec_in, dec_out

In [10]:
# sample = [['wood', '나무'], ['word', '단어']]
# encoding(sample)

In [11]:
# 6. 전체 번역 데이터 처리
X_enc, X_dec, Y_dec = encoding(eng_kor)

# numpy로 전환
X_enc = np.array(X_enc)
X_dec = np.array(X_dec)
Y_dec = np.array(Y_dec)

X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3))

In [12]:
# 7. 모델 구현
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 출력, state_c, state_h
_, state_h, state_c = LSTM(units=MY_HIDDEN,  # 128
                           return_state=True)(ENC_IN)
# 인코더와 디코더 연결
link = [state_h, state_c]

# 디코더 구현
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
               return_sequences=True)(DEC_IN, initial_state=link)  # link:파라미터 결과값 받기
# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size, activation='softmax')(DEC_MID)

# 모델 구조 구현
model = Model(inputs=[ENC_IN, DEC_IN], outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        153600      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

In [13]:
# 8. 학습과정 설정 & 학습하기
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
begin = time()
model.fit([X_enc, X_dec], Y_dec, epochs=MY_EPOCH, verbose=1)
end = time()
print('학습시간 : ', end-begin)

Epoch 1/500
4/4 [==============================] - 4s 11ms/step - loss: 5.1014 - accuracy: 0.2364
Epoch 2/500
4/4 [==============================] - 0s 9ms/step - loss: 4.8522 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 11ms/step - loss: 3.7856 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 9ms/step - loss: 3.4418 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 11ms/step - loss: 3.3983 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 7ms/step - loss: 3.3691 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 11ms/step - loss: 3.3449 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 8ms/step - loss: 3.3199 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 9ms/step - loss: 3.2972 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 13ms/step - loss: 3.2798 - accuracy: 0.3333
Epoch 11/500
4/4 [======

4/4 [==============================] - 0s 11ms/step - loss: 1.2201 - accuracy: 0.8333
Epoch 85/500
4/4 [==============================] - 0s 7ms/step - loss: 1.1774 - accuracy: 0.8485
Epoch 86/500
4/4 [==============================] - 0s 10ms/step - loss: 1.1533 - accuracy: 0.8455
Epoch 87/500
4/4 [==============================] - 0s 12ms/step - loss: 1.1071 - accuracy: 0.8515
Epoch 88/500
4/4 [==============================] - 0s 10ms/step - loss: 1.0946 - accuracy: 0.8576
Epoch 89/500
4/4 [==============================] - 0s 7ms/step - loss: 1.0861 - accuracy: 0.8697
Epoch 90/500
4/4 [==============================] - 0s 9ms/step - loss: 1.0061 - accuracy: 0.9000
Epoch 91/500
4/4 [==============================] - 0s 6ms/step - loss: 0.9808 - accuracy: 0.9000
Epoch 92/500
4/4 [==============================] - 0s 10ms/step - loss: 0.9533 - accuracy: 0.9061
Epoch 93/500
4/4 [==============================] - 0s 7ms/step - loss: 0.9393 - accuracy: 0.9091
Epoch 94/500
4/4 [==========

4/4 [==============================] - 0s 12ms/step - loss: 0.0422 - accuracy: 1.0000
Epoch 167/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0398 - accuracy: 1.0000
Epoch 168/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0360 - accuracy: 1.0000
Epoch 169/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0349 - accuracy: 1.0000
Epoch 170/500
4/4 [==============================] - 0s 7ms/step - loss: 0.0342 - accuracy: 1.0000
Epoch 171/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0318 - accuracy: 1.0000
Epoch 172/500
4/4 [==============================] - 0s 10ms/step - loss: 0.0326 - accuracy: 0.9970
Epoch 173/500
4/4 [==============================] - 0s 5ms/step - loss: 0.0303 - accuracy: 0.9970
Epoch 174/500
4/4 [==============================] - 0s 9ms/step - loss: 0.0305 - accuracy: 0.9970
Epoch 175/500
4/4 [==============================] - 0s 11ms/step - loss: 0.0256 - accuracy: 1.0000
Epoch 176/500
4/4 [=

4/4 [==============================] - 0s 10ms/step - loss: 4.4179e-04 - accuracy: 1.0000
Epoch 249/500
4/4 [==============================] - 0s 11ms/step - loss: 4.5941e-04 - accuracy: 1.0000
Epoch 250/500
4/4 [==============================] - 0s 9ms/step - loss: 4.0631e-04 - accuracy: 1.0000
Epoch 251/500
4/4 [==============================] - 0s 11ms/step - loss: 7.4970e-04 - accuracy: 1.0000
Epoch 252/500
4/4 [==============================] - 0s 6ms/step - loss: 0.0050 - accuracy: 0.9970
Epoch 253/500
4/4 [==============================] - 0s 10ms/step - loss: 3.8701e-04 - accuracy: 1.0000
Epoch 254/500
4/4 [==============================] - 0s 12ms/step - loss: 3.4916e-04 - accuracy: 1.0000
Epoch 255/500
4/4 [==============================] - 0s 7ms/step - loss: 3.2226e-04 - accuracy: 1.0000
Epoch 256/500
4/4 [==============================] - 0s 10ms/step - loss: 3.0002e-04 - accuracy: 1.0000
Epoch 257/500
4/4 [==============================] - 0s 8ms/step - loss: 2.8019e-04 -

4/4 [==============================] - 0s 7ms/step - loss: 2.0611e-05 - accuracy: 1.0000
Epoch 328/500
4/4 [==============================] - 0s 10ms/step - loss: 5.3705e-04 - accuracy: 1.0000
Epoch 329/500
4/4 [==============================] - 0s 8ms/step - loss: 6.7640e-05 - accuracy: 1.0000
Epoch 330/500
4/4 [==============================] - 0s 6ms/step - loss: 2.0907e-05 - accuracy: 1.0000
Epoch 331/500
4/4 [==============================] - 0s 11ms/step - loss: 1.6924e-05 - accuracy: 1.0000
Epoch 332/500
4/4 [==============================] - 0s 7ms/step - loss: 1.4458e-05 - accuracy: 1.0000
Epoch 333/500
4/4 [==============================] - 0s 9ms/step - loss: 1.2662e-05 - accuracy: 1.0000
Epoch 334/500
4/4 [==============================] - 0s 11ms/step - loss: 1.0228e-05 - accuracy: 1.0000
Epoch 335/500
4/4 [==============================] - 0s 8ms/step - loss: 9.5984e-06 - accuracy: 1.0000
Epoch 336/500
4/4 [==============================] - 0s 9ms/step - loss: 9.0381e-06 

4/4 [==============================] - 0s 8ms/step - loss: 1.4562e-06 - accuracy: 1.0000
Epoch 407/500
4/4 [==============================] - 0s 6ms/step - loss: 1.4251e-06 - accuracy: 1.0000
Epoch 408/500
4/4 [==============================] - 0s 12ms/step - loss: 1.4323e-06 - accuracy: 1.0000
Epoch 409/500
4/4 [==============================] - 0s 12ms/step - loss: 1.4143e-06 - accuracy: 1.0000
Epoch 410/500
4/4 [==============================] - 0s 9ms/step - loss: 1.3633e-06 - accuracy: 1.0000
Epoch 411/500
4/4 [==============================] - 0s 10ms/step - loss: 1.3507e-06 - accuracy: 1.0000
Epoch 412/500
4/4 [==============================] - 0s 11ms/step - loss: 1.3435e-06 - accuracy: 1.0000
Epoch 413/500
4/4 [==============================] - 0s 6ms/step - loss: 1.3135e-06 - accuracy: 1.0000
Epoch 414/500
4/4 [==============================] - 0s 10ms/step - loss: 1.3109e-06 - accuracy: 1.0000
Epoch 415/500
4/4 [==============================] - 0s 6ms/step - loss: 1.2846e-0

4/4 [==============================] - 0s 7ms/step - loss: 6.6071e-07 - accuracy: 1.0000
Epoch 486/500
4/4 [==============================] - 0s 9ms/step - loss: 6.6179e-07 - accuracy: 1.0000
Epoch 487/500
4/4 [==============================] - 0s 5ms/step - loss: 6.5168e-07 - accuracy: 1.0000
Epoch 488/500
4/4 [==============================] - 0s 12ms/step - loss: 6.5240e-07 - accuracy: 1.0000
Epoch 489/500
4/4 [==============================] - 0s 11ms/step - loss: 6.4626e-07 - accuracy: 1.0000
Epoch 490/500
4/4 [==============================] - 0s 6ms/step - loss: 6.4301e-07 - accuracy: 1.0000
Epoch 491/500
4/4 [==============================] - 0s 10ms/step - loss: 6.3650e-07 - accuracy: 1.0000
Epoch 492/500
4/4 [==============================] - 0s 8ms/step - loss: 6.2964e-07 - accuracy: 1.0000
Epoch 493/500
4/4 [==============================] - 0s 9ms/step - loss: 6.2206e-07 - accuracy: 1.0000
Epoch 494/500
4/4 [==============================] - 0s 10ms/step - loss: 6.2061e-07

In [14]:
# 9. 모델 사용하기
test = [['cold', 'PP'],
        ['down', 'PP'],
        ['desk', 'PP'],
        ['love', 'PP'],
        ['olve', 'PP'],
        ['lvoe', 'PP'],
        ['loev', 'PP'],
        ['eovl', 'PP']]
enc_in, dec_in, dec_out = encoding(test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((8, 4, 171), (8, 3, 171))

In [15]:
pred = model.predict([enc_in, dec_in])
pred

1/1 [==============================] - 1s 970ms/step


array([[[8.82762152e-15, 2.54920252e-18, 9.08677549e-15, ...,
         1.27003247e-12, 1.04847610e-10, 3.21153308e-08],
        [3.03709614e-15, 2.11162487e-09, 1.86043884e-15, ...,
         2.59950883e-09, 8.09314042e-13, 4.44297654e-11],
        [4.57945960e-20, 1.00000000e+00, 3.33417376e-20, ...,
         1.07257385e-14, 5.90280262e-18, 3.22687888e-17]],

       [[1.56262739e-14, 2.09701561e-16, 2.55178687e-14, ...,
         4.73608186e-13, 3.88113180e-13, 7.14939929e-08],
        [1.34387957e-13, 2.59253653e-07, 7.62183082e-14, ...,
         1.50365873e-10, 1.87443021e-14, 5.69284175e-10],
        [6.42138126e-17, 9.99999881e-01, 5.36256393e-17, ...,
         4.30270836e-13, 1.13920191e-17, 2.36454160e-15]],

       [[5.71454437e-14, 6.14890968e-16, 7.80122291e-14, ...,
         3.40798809e-16, 1.09650644e-08, 9.54342248e-12],
        [1.70245382e-13, 7.22934743e-08, 2.65549139e-13, ...,
         5.82387524e-12, 2.30452853e-12, 2.72322130e-11],
        [7.07066788e-16, 1.00000000e

In [16]:
pred.shape  # (8:test, 3:출력, 171)

(8, 3, 171)

In [17]:
# alpha[32], char_to_num['감']
# 출력 형식 : cold -> 감기(32, xx, 1)
# test[i][0]  # 문제
for i in range(len(pred)):
    eng = test[i][0]
    hat = pred[i].argmax(axis=1)  # array
    kor = alpha[hat[0]] + alpha[hat[1]]
    print('{} -> {} {}'.format(eng, kor, hat))

cold -> 감기 [32 46  1]
down -> 아래 [111  67   1]
desk -> 책상 [149  97   1]
love -> 사랑 [96 66  1]
olve -> 사랑 [96 66  1]
lvoe -> 사랑 [96 66  1]
loev -> 사랑 [96 66  1]
eovl -> 도E [62  1  1]
